In [1]:
import pandas as pd
import re
import concurrent.futures
import requests
import openai
from tqdm import tqdm
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter

In [2]:
OPENAI_API_KEY = ''

In [3]:
## clean and process so

def cleanhtml(raw_html):
    CLEANR = re.compile('<.*?>')
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext
    
def clean_data(df):
    df["pt_answer"] = df["pt_answer"].apply(lambda x: cleanhtml(x))

    df["question"] = df["pt_title"].str.lower() + "\n" + df["pt_body"]
    df["question"] = df["question"].apply(lambda x: cleanhtml(x))
    df["answer"] = df["pt_answer"].str.lower()

    df = df[['pt_post_id','question', 'answer']]
    return df
    

## get qa and link to post
def get_url(df):
    url = []
    for index, row in df.iterrows():
        url.append(f"https://stackoverflow.com/questions/{row['pt_post_id']}/")
    
    df['source'] = url

    return df


In [4]:

df = pd.read_csv('pt_question_answers_updated.csv')
df.shape

(10763, 12)

In [5]:
df = clean_data(df)
df = get_url(df)
df = df[['question','answer','source']]

In [8]:
df = df[:5000]

df.shape

(5000, 3)

## using openai summarize accepted answer

In [9]:
def get_qa_openai(context, index):
    try:
        completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", api_key = OPENAI_API_KEY,
                messages=[
                    {"role": "user", "content": context}
                  ]
                )

        qa = completion.choices[0].message.content

    except requests.exceptions.RequestException as e:
        print(f'Request failed with error: {str(e)}.')
        print(f'Waiting for 3 minutes before trying again...')
        time.sleep(180)
    
    return (qa, index)

In [10]:
questions_ans = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    
    futures = []
    for index,i in df.iterrows():

        context = f"Given a pytorch question and answer given below, How will an expert PyTorch engineer answer this question? Include code as appropriate and do not mentioned your role in the answer \
                    question: {i['question']}, answer:{i['answer']}"
        
        futures.append(executor.submit(get_qa_openai, context, index))

    for future, (_, row) in tqdm(zip(concurrent.futures.as_completed(futures), df.iterrows()), total=len(df)):
        try:
            qa, ind = future.result()
            questions_ans.append((ind,qa))
        except Exception as exc:
            print(f'generated an exception: {exc}')

  1%|██▎                                                                                                                                                                  | 71/5000 [00:50<28:41,  2.86it/s]

generated an exception: This model's maximum context length is 4097 tokens. However, your messages resulted in 4728 tokens. Please reduce the length of the messages.


 17%|███████████████████████████▌                                                                                                                                        | 839/5000 [10:25<57:44,  1.20it/s]

generated an exception: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 30eae6c29b6ef042f1d954015dd11a2f in your message.)


 18%|█████████████████████████████▉                                                                                                                                      | 914/5000 [11:18<21:56,  3.10it/s]

...

generated an exception: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b872cab101ebfd425b86960b7a2ce36b in your message.)


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████    | 4878/5000 [1:02:03<01:01,  1.97it/s]

generated an exception: This model's maximum context length is 4097 tokens. However, your messages resulted in 4360 tokens. Please reduce the length of the messages.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [1:05:08<00:00,  1.28it/s]

generated an exception: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 18 May 2023 03:38:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7c9109fad996c561-SEA', 'alt-svc': 'h3=":443"; ma=86400, h3-29=":443"; ma=86400'}


In [11]:
for index, qa in questions_ans:
    df.at[index, 'short_answer'] = qa

In [12]:
df

,question,answer,source,short_answer
0,extracting the top-k value-indices from a 1-d ...,as of pull request #496 torch now includes a b...,https://stackoverflow.com/questions/34750268/,An expert PyTorch engineer would suggest using...
1,how to display custom images in tensorboard (e...,it is quite easy to do if you have the image i...,https://stackoverflow.com/questions/38543850/,"As an expert PyTorch engineer, I can suggest a..."
2,python wheels: cp27mu not supported\nI'm tryin...,this is exactly that. \nrecompile python under...,https://stackoverflow.com/questions/41767005/,Code to recompile Python with --enable-unicode...
3,loading torch7 trained models (.t7) in pytorch...,as of pytorch 1.0 torch.utils.serialization is...,https://stackoverflow.com/questions/41861354/,An expert PyTorch engineer would first acknowl...
4,pytorch: how to use dataloaders for custom dat...,"yes, that is possible. just create the objects...",https://stackoverflow.com/questions/41924453/,An expert PyTorch engineer would answer by pro...
...,...,...,...,...
4995,compare two segmentation maps predictions\nI a...,"yes, this is a valid way to implement consiste...",https://stackoverflow.com/questions/67682106/,Regarding the computation of the total_supervi...
4996,difference between dataset and tensordataset i...,the dataset class is an abstract class that is...,https://stackoverflow.com/questions/67683406/,An expert PyTorch engineer would explain that ...
4997,define nn.parameters with a for loop\nI am int...,best way to accomplish this you can accomplish...,https://stackoverflow.com/questions/67689104/,An expert PyTorch engineer would suggest using...
4998,how to integrate a pytorch model into a dynami...,tensorflow or pytorch models can't be directly...,https://stackoverflow.com/questions/67693181/,"As of now, direct integration of PyTorch model..."


In [13]:
df.to_csv('so_5k_with_short_answer.csv')

## data cleaning

In [14]:
df.isna().sum()

question         0
answer           0
source           0
short_answer    60
dtype: int64

In [15]:
df = df.dropna()

In [16]:
df

,question,answer,source,short_answer
0,extracting the top-k value-indices from a 1-d ...,as of pull request #496 torch now includes a b...,https://stackoverflow.com/questions/34750268/,An expert PyTorch engineer would suggest using...
1,how to display custom images in tensorboard (e...,it is quite easy to do if you have the image i...,https://stackoverflow.com/questions/38543850/,"As an expert PyTorch engineer, I can suggest a..."
2,python wheels: cp27mu not supported\nI'm tryin...,this is exactly that. \nrecompile python under...,https://stackoverflow.com/questions/41767005/,Code to recompile Python with --enable-unicode...
3,loading torch7 trained models (.t7) in pytorch...,as of pytorch 1.0 torch.utils.serialization is...,https://stackoverflow.com/questions/41861354/,An expert PyTorch engineer would first acknowl...
4,pytorch: how to use dataloaders for custom dat...,"yes, that is possible. just create the objects...",https://stackoverflow.com/questions/41924453/,An expert PyTorch engineer would answer by pro...
...,...,...,...,...
4995,compare two segmentation maps predictions\nI a...,"yes, this is a valid way to implement consiste...",https://stackoverflow.com/questions/67682106/,Regarding the computation of the total_supervi...
4996,difference between dataset and tensordataset i...,the dataset class is an abstract class that is...,https://stackoverflow.com/questions/67683406/,An expert PyTorch engineer would explain that ...
4997,define nn.parameters with a for loop\nI am int...,best way to accomplish this you can accomplish...,https://stackoverflow.com/questions/67689104/,An expert PyTorch engineer would suggest using...
4998,how to integrate a pytorch model into a dynami...,tensorflow or pytorch models can't be directly...,https://stackoverflow.com/questions/67693181/,"As of now, direct integration of PyTorch model..."


In [18]:
df['q_len'] = df['question'].str.len()
df['c_len'] = df['answer'].str.len()
df['a_len'] = df['short_answer'].str.len()
df["total"] = df['q_len'] + df['c_len'] + df['a_len']

df

/tmp/ipykernel_2766/3823359095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['q_len'] = df['question'].str.len()
/tmp/ipykernel_2766/3823359095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_len'] = df['answer'].str.len()
/tmp/ipykernel_2766/3823359095.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,question,answer,source,short_answer,q_len,c_len,a_len,total
0,extracting the top-k value-indices from a 1-d ...,as of pull request #496 torch now includes a b...,https://stackoverflow.com/questions/34750268/,An expert PyTorch engineer would suggest using...,359,788,473,1620
1,how to display custom images in tensorboard (e...,it is quite easy to do if you have the image i...,https://stackoverflow.com/questions/38543850/,"As an expert PyTorch engineer, I can suggest a...",599,1020,1095,2714
2,python wheels: cp27mu not supported\nI'm tryin...,this is exactly that. \nrecompile python under...,https://stackoverflow.com/questions/41767005/,Code to recompile Python with --enable-unicode...,2110,113,687,2910
3,loading torch7 trained models (.t7) in pytorch...,as of pytorch 1.0 torch.utils.serialization is...,https://stackoverflow.com/questions/41861354/,An expert PyTorch engineer would first acknowl...,676,516,1112,2304
4,pytorch: how to use dataloaders for custom dat...,"yes, that is possible. just create the objects...",https://stackoverflow.com/questions/41924453/,An expert PyTorch engineer would answer by pro...,298,1443,2517,4258
...,...,...,...,...,...,...,...,...
4995,compare two segmentation maps predictions\nI a...,"yes, this is a valid way to implement consiste...",https://stackoverflow.com/questions/67682106/,Regarding the computation of the total_supervi...,2830,421,194,3445
4996,difference between dataset and tensordataset i...,the dataset class is an abstract class that is...,https://stackoverflow.com/questions/67683406/,An expert PyTorch engineer would explain that ...,251,854,600,1705
4997,define nn.parameters with a for loop\nI am int...,best way to accomplish this you can accomplish...,https://stackoverflow.com/questions/67689104/,An expert PyTorch engineer would suggest using...,1300,723,1019,3042
4998,how to integrate a pytorch model into a dynami...,tensorflow or pytorch models can't be directly...,https://stackoverflow.com/questions/67693181/,"As of now, direct integration of PyTorch model...",1410,914,930,3254


In [34]:
## checking sample data

for i in range(5):
    print("*" * 100)
    print("!!!! Question: ", df.loc[i, "question"])
    print("!!!! Context: ", df.loc[i, "answer"])
    print("!!!! Answer: ", df.loc[i, "short_answer"])


****************************************************************************************************
!!!! Question:  extracting the top-k value-indices from a 1-d tensor
Given a 1-D tensor in Torch (torch.Tensor), containing values which can be compared (say floating point), how can we extract the indices of the top-k values in that tensor?
Apart from the brute-force method, I am looking for some API call, that Torch/lua provides, which can perform this task efficiently.

!!!! Context:  as of pull request #496 torch now includes a built-in api named torch.topk. example:

&gt; t = torch.tensor{9, 1, 8, 2, 7, 3, 6, 4, 5}

-- obtain the 3 smallest elements
&gt; res = t:topk(3)
&gt; print(res)
 1
 2
 3
[torch.doubletensor of size 3]

-- you can also get the indices in addition
&gt; res, ind = t:topk(3)
&gt; print(ind)
 2
 4
 6
[torch.longtensor of size 3]

-- alternatively you can obtain the k largest elements as follow
-- (see the api documentation for more details)
&gt; res = t:topk(3, t

## preparing alpaca lora dataset format

In [29]:

import json
data = []

for index, row in df.iterrows():
    d = { "instruction": row['question'],
          "input": row['answer'],
          "output": row['short_answer']
        }
    
    data.append(d)
    

with open('so_5k_with_short_answer.json', 'w') as f:
    json.dump(data, f)